In [ ]:
# runtime -> change Runtime Type -> T4 so training doesn't become O(n!) LOL

# In terminal, run:
    # nvidia-smi: see what GPU
    # pip install ultralytics: gotta install from scratch each session I think

# 'mount'/connect google drive
# Folder icon -> folder with Google Drive (give access)
from google.colab import drive
drive.mount('/content/drive')

import zipfile
import os

zip_path = "/content/drive/MyDrive/Colab_Notebooks/drone-thermal-dataset.zip"
extract_path = "/content/drive/MyDrive/Colab_Notebooks/drone-thermal-dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset extracted!


In [ ]:
# In terminal, run:
    # cd /content/drive/MyDrive/Colab_Notebooks/drone-thermal-detection/drone-thermal-detection
import zipfile
import os
import yaml

# ! bash command, don't forget!
zip_path = "/content/drive/MyDrive/Colab_Notebooks/drone-thermal-dataset.zip" # takes from drive
extract_path = "/content/local_dataset" # saves 'locally' on server so doesn't interface with Google Drive, save

print(f"Extracting {zip_path} to {extract_path}...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print("Extraction complete!")

# chat said it fixes YAML sourcing, it does, no more questions
yaml_file = f"{extract_path}/drone-thermal-dataset/data.yaml"

with open(yaml_file, 'r') as f:
    config = yaml.safe_load(f)

# Force paths to be correct for this session
config['path'] = f"{extract_path}/drone-thermal-dataset"
config['train'] = "train/images"
config['val'] = "val/images"

with open(yaml_file, 'w') as f:
    yaml.dump(config, f)

print("YAML file paths patched successfully!")


from ultralytics import YOLO

model = YOLO('yolo11n.pt') # yolo11 Nano

# epoch checkpoint callback
def on_train_epoch_end(trainer):
    current_epoch = trainer.epoch + 1
    total_epochs = trainer.epochs
    print(f"\n Epoch {current_epoch}/{total_epochs} Complete. Checkpoint 'last.pt' saved to Drive.")

# Register the callback
model.add_callback("on_train_epoch_end", on_train_epoch_end)

# train
results = model.train(
    data=f"{extract_path}/drone-thermal-dataset/data.yaml", # config file with classes
    task='detect',

    epochs=100,
    imgsz=960,       # quarter resolution, multiple of 32, big enough hopefully
    batch=6,         # prevent out of memory cause large resolution
    patience=20,     # check if diminishing returns every 20 epochs
    device=0,        # Use GPU 0
    workers=4,        # chat said it parallelizes faster, no questions

    project='/content/drive/MyDrive/Colab_Notebooks/Drone_Thermal_Run', # checkpoint to Google Drive
    name='yolo11n_960px_v14',

    # --- Thermal-Specific Augmentations (The "Secret Sauce") ---
    hsv_h=0.0,       # blue for heat messes with hue
    hsv_s=0.8,       # saturation turns more blue, making heat signature color gradient steeper, easier to track
    hsv_v=0.4,       # standard brightness

    # --- Geometry Augmentations ---
    fliplr=0.5,      # left/right flip fine
    flipud=0,        # I'd assume this would help it be vertical agnostic, but ig not
    degrees=10.0,    # allows for rotation due to drone acceleration

    # simulates distortion of viewing flat ground at 45 degree angle,
    perspective=0.0005, # helps with 'far vs near' object shape changes

    # helps simulate altitude changes
    scale=0.5,  # scale/zoom in/out to simulate altitude cahnge

    # --- Plotting ---
    plots=True       # Automatically generate loss curves
)

print("Training Complete!")


Extracting /content/drive/MyDrive/Colab_Notebooks/drone-thermal-dataset.zip to /content/local_dataset...
Extraction complete!
YAML file paths patched successfully!
Ultralytics 8.4.12 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=6, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/local_dataset/drone-thermal-dataset/data.yaml, degrees=10.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0, hsv_s=0.8, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, 